# Admin Steps

In [ ]:
# We install these packages first to allow connectivity to the remote database

!pip install sqlalchemy
!pip install PyMySQL

: 

## STEP 1 Use SQL to successfully retrieve dataset from remote Heicoders database, using the code below.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine



ENDPOINT = 'heicoders-playground.c2ced10ceyki.ap-southeast-1.rds.amazonaws.com'
PORT = 3306
USERNAME = 'student300'
PASSWORD = 'heicoders_AI300'
DBNAME = 'ai300_capstone'

database_conn = create_engine(f'mysql+pymysql://{USERNAME}:{PASSWORD}@{ENDPOINT}/{DBNAME}')

## STEP 2 Use SQL to retrieve at least one column

## To verify columns against data dictionary.

query_account = """
    SELECT * FROM account;

"""

query_account_usage = """
    SELECT * FROM account_usage;

"""

query_churn = """
    SELECT * FROM churn_status;

"""

query_customer = """
    SELECT * FROM customer;

"""

query_city = """
    SELECT * FROM city;

"""

df_1 = pd.read_sql(query_account, database_conn)
df_1

In [ ]:
df_2 = pd.read_sql(query_account_usage, database_conn)
df_2

In [ ]:
df_3 = pd.read_sql(query_churn, database_conn)
df_3

In [ ]:
df_4 = pd.read_sql(query_customer, database_conn)
df_4

In [ ]:
df_5 = pd.read_sql(query_city, database_conn)
df_5

## STEP 2 - Extract at least one column per table.

In [ ]:


query_account_col = """
    SELECT account_id, tenure_months FROM account;

"""

query_account_usage_col = """
    SELECT stream_tv,stream_movie FROM account_usage;

"""

query_churn_col = """
    SELECT churn_label, churn_category FROM churn_status;

"""

query_customer_col = """
    SELECT gender,age,married FROM customer;

"""

query_city_col = """
    SELECT area_id, zip_code, city FROM city;

"""

df_6 = pd.read_sql(query_account_col, database_conn)
df_7 = pd.read_sql(query_account_usage_col, database_conn)
df_8 = pd.read_sql(query_churn_col, database_conn)
df_9 = pd.read_sql(query_customer_col, database_conn)
df_10 = pd.read_sql(query_city_col, database_conn)
df_6

In [ ]:
df_7

In [ ]:
df_8

In [ ]:
df_9

In [ ]:
df_10

In [ ]:
# To merge all tables

merged_churn = """
    Select 
    b.account_id, b.customer_id, b.tenure_months, b.num_referrals, 
    b.has_internet_service, b.internet_type,b.has_unlimited_data,b.has_phone_service, 
    b.has_multiple_lines, b.has_premium_tech_support, b.has_online_security, b.has_online_backup, 
    b.has_device_protection, b.contract_type, b.paperless_billing, b.payment_method,
    a.status, a.churn_label, a.churn_category,a.churn_reason,
    c.avg_long_distance_fee_monthly,c.total_long_distance_fee,c.avg_gb_download_monthly,
    c.stream_tv,c.stream_movie,c.stream_music,c.total_monthly_fee,c.total_charges_quarter,c.total_refunds,
    d.gender,d.age,d.senior_citizen,d.married,d.num_dependents,
    e.area_id,e.zip_code,e.city,e.latitutde,e.longitude,e.population
    From churn_status as a
    Left join account as b On a.customer_id = b.customer_id
    Left join account_usage as c On b.account_id = c.account_id
    Left join customer as d On a.customer_id = d.customer_id
    Left join city as e On d.zip_code = e.zip_code;

"""

df_churn_table = pd.read_sql(merged_churn, database_conn)
df_churn_table


## STEP 3 Visualise the data using matplotlib/plotly

In [ ]:
#import visualisation packages like plotly and matplotlib here

import plotly.express as px
import seaborn as sbn
import matplotlib.pyplot as plt

!pip install folium

import folium

In [ ]:
## Graph 1: I hypothesised that we should not expect people <65 years old to churn credit cards
## as they are still working and need credit cards to manage cash flow. 

sbn.histplot(df_churn_table, x='age',hue='status')

## Notice in the histogram that people approaching and ust past the age of 50 years old is churning more than the other groups.

In [ ]:
## Graph 2: I also hypothesised that those who will churn will likely have a shorter tenure duration, 
## and they would have spent less than those who stayed as they are likely opportunistic in pursuing promotions elsewhere.

fig = px.histogram(df_churn_table, 
                   x = 'tenure_months',
                   y = 'total_long_distance_fee',
                   color = 'status',
                   marginal = 'box' # or violin, rug
)

fig.show()

In [ ]:
## Graph X: I hypothesised that people living in specific parts of the country could be suggesitble to churn due to
## cultural norms and expectations. So I crafted

print(df_churn_table['longitude'].min())
print(df_churn_table['longitude'].max())
print(df_churn_table['latitutde'].min())
print(df_churn_table['latitutde'].max())



In [17]:
# Code below used to make a scatterplot visualisation that yielded no meaningful trend and only clutter. 

# To save if code is needed again.Avoid scatterplots in the meanwhile.


# ## fig = px.scatter(df_churn_table,
#                  x='age',
#                  y='tenure_months',
#                  color='gender'
# )

# fig.update_yaxes(title_text="tenure_months", ticksuffix=" months")
# fig.update_xaxes(title_text="Age", ticksuffix=" years")

# fig.show()

## STEP 4 Perform Feature Engineering on the dataset

In [18]:
#importing packages for this segment

import numpy as np
from sklearn.preprocessing import LabelEncoder


In [ ]:
df_churn_table.isnull().sum()

## Null cell check round 1, but it looks like there is nothing despite there being null cells above.

In [ ]:
(df_churn_table == '').sum()

## Discovered empty string cells in churn_label, churn_category and churn_reason

In [ ]:
df_churn_table.info()

In [ ]:
df_churn_table.describe()

In [ ]:
print(df_3.loc[df_3["churn_reason"] == ""].head(50))
print(df_3.loc[df_3["churn_reason"] == ""].tail(50))
print(df_3.loc[df_3["churn_category"] == ""].head(50))
print(df_3.loc[df_3["churn_category"] == ""].tail(50))

In [ ]:
df_3.loc[df_3["churn_label"] == ""]

In [ ]:
df_3.loc[df_3["churn_label"] == ""]

In [ ]:
## Looking further at the columns with empty string, we decided to drop some rows just for "churn_label".
## This is because the reasons for churning are beyond the company's control. 
## We decided to ignore cleaning "churn_category" and "churn_reason" as the vast majority (>30%) of cells are empty. 


##Dropping the columns

df_churn_revised = df_churn_table.drop(columns=['churn_category','churn_reason'])


##Dropping rows for blank churn_labels
df_churn_clean =df_churn_revised[df_churn_revised['churn_label'] != ""]
df_churn_revised[df_churn_revised['churn_label'] != ""]
df_churn_clean



In [ ]:
## We will keep the following fields unchanged:
## account - account_id, customer_id, tenure_months, num_referrals
## account_usage - account_id, avg_long_distance, total_long_distance_fee, avg_gb_download_monthly, 
## account_usage - total_monthly_fee, total_chargers_quarter, total_refunds
## Churn_status - customer_id, churn_category (removed), churn_reason (removed)
## customer - Customer_id, age, num_dependents, zip_code
## city - area_id, zip_code, latitude, longitutde, population

## Using label encoding by table of origin for variables with only two options.

## Account

df_churn_clean2 = df_churn_clean.copy()

cols = ['has_internet_service','has_unlimited_data','has_phone_service','has_multiple_lines','has_premium_tech_support'
       ,'has_online_security','has_online_backup','has_device_protection','paperless_billing','stream_tv',
        'stream_movie','stream_music','churn_label','gender','senior_citizen','married']

for col in cols:
    df_churn_clean2[col] = df_churn_clean[col].replace({'Yes':1,'No':0})
    
df_churn_clean2['gender'] = df_churn_clean['gender'].replace({'Male':1,'Female':0})

# df_churn_clean['has_internet_service'] = label_encoder.fit_transform(df_churn_clean['has_internet_service'])
# df_churn_clean['has_unlimited_data'] = label_encoder.fit_transform(df_churn_clean['has_unlimited_data'])
# df_churn_clean['has_phone_service'] = label_encoder.fit_transform(df_churn_clean['has_phone_service'])
# df_churn_clean['has_multiple_lines'] = label_encoder.fit_transform(df_churn_clean['has_multiple_lines'])
# df_churn_clean['has_premium_tech_support'] = label_encoder.fit_transform(df_churn_clean['has_premium_tech_support'])
# df_churn_clean['has_online_security'] = label_encoder.fit_transform(df_churn_clean['has_online_security'])
# df_churn_clean['has_online_backup'] = label_encoder.fit_transform(df_churn_clean['has_online_backup'])
# df_churn_clean['has_device_protection'] = label_encoder.fit_transform(df_churn_clean['has_device_protection'])
# df_churn_clean['paperless_billing'] = label_encoder.fit_transform(df_churn_clean['paperless_billing'])

# Has_internet_service – change to boolean (1,0)
# Has_unlimited_data – change to boolean (1,0)
# Has_phone_service – change to boolean (1,0)
# Has_multiple_lines – change to boolean (1,0)
# Has_premium_tech_support – change to boolean (1,0)
# Has_online_security – change to boolean (1,0)
# Has_online_backup – change to boolean (1,0)
# Has_device_protection – change to boolean (1,0)
# paperless_billing – change to boolean (1,0)

## Account_usage

# df_churn_clean['stream_tv'] = label_encoder.fit_transform(df_churn_clean['stream_tv'])
# df_churn_clean['stream_movie'] = label_encoder.fit_transform(df_churn_clean['stream_movie'])
# df_churn_clean['stream_music'] = label_encoder.fit_transform(df_churn_clean['stream_music'])

# Stream_tv - change to boolean (1,0)
# Stream_movie - change to boolean (1,0)
# Stream_music - change to boolean (1,0)


# ## Churn_status
# df_churn_clean['churn_label'] = label_encoder.fit_transform(df_churn_clean['churn_label'])
# # Churn_label – change to boolean

# # ## Customer
# df_churn_clean['gender'] = label_encoder.fit_transform(df_churn_clean['gender'])
# df_churn_clean['senior_citizen'] = label_encoder.fit_transform(df_churn_clean['senior_citizen'])
# df_churn_clean['married'] = label_encoder.fit_transform(df_churn_clean['married'])
# Gender - change to boolean (1,0)
# Senior_citizen - change to boolean (1,0)
# married - change to boolean (1,0)

df_churn_clean

In [ ]:
df_churn_clean2

In [29]:
## Using one-hot encoding by table of origin for variables with more than 2 answers

# Create a LabelEncoder object

## Account
internet_type_OH = pd.get_dummies(df_churn_clean['internet_type'],dtype=int)
contract_type_OH = pd.get_dummies(df_churn_clean['contract_type'],dtype=int)
payment_method_OH = pd.get_dummies(df_churn_clean['payment_method'],dtype=int)


# Internet type – change to categorical (1,2,3,4)
# Contract_type – change to categorical (1,2,3)
# Payment_ method – change to categorical (1,2,3)

## Account_usage - None

## Churn_status

status_OH = pd.get_dummies(df_churn_clean['status'],dtype=int)



# status – change to categorical (0 for joined, 1 for stayed, 2 for leave)

# City Label encoding due to 'object' variable

label_encoder = LabelEncoder()
df_churn_clean2['city'] = label_encoder.fit_transform(df_churn_clean2['city'])

## Customer - None

df_fin_churn = pd.concat([df_churn_clean2,internet_type_OH,contract_type_OH,payment_method_OH,status_OH], axis=1)
df_fin_churn.drop(['internet_type','contract_type','payment_method','status'],axis='columns',inplace=True)

In [ ]:
df_fin_churn.columns

In [ ]:
df_fin_churn.info()

In [ ]:
corr = df_fin_churn.corr(numeric_only=True)
fig = px.imshow(corr, color_continuous_scale = 'Brwnyl', text_auto = True)
fig.update_xaxes(side="top")
fig.show()

In [ ]:
df_fin_churn['churn_label']

In [ ]:
df_fin_churn['city']

## STEP 5-7 CREATE A MODEL FOR LOGISTIC REGRESSION using train test split and GridsearchCV to find the best parameters

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statistics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import boxcox
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statistics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_auc_score

In [ ]:
## field test for hyperparameter tuning - XGB

features= df_fin_churn.columns.drop(['churn_label','account_id','customer_id'])

X = df_fin_churn[['tenure_months','has_unlimited_data','has_premium_tech_support','has_device_protection','total_monthly_fee']]
y = df_fin_churn['churn_label'].values

xgb_model = XGBClassifier(random_state = 5)

search_space = {
    "n_estimators": [100,200,500],
    "max_depth" : [3, 6, 9],
    'gamma' : [0.01, 0.1],
    "learning_rate": [0.001, 0.01, 0.1, 1]
}

GS = GridSearchCV(estimator = xgb_model,
            param_grid = search_space,
            scoring = 'roc_auc',
             cv = 5,
             verbose = 4            
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

GS.fit(X_train,y_train)

In [ ]:
print(GS.best_estimator_)

In [ ]:
print(GS.best_params_)

In [ ]:
print(GS.best_score_)

In [ ]:
# Using Logistic Regression

log_reg = LogisticRegression(max_iter=300)


# Split data into predictors X and output Y

features= df_fin_churn.columns.drop(['churn_label','account_id','customer_id'])


X = df_fin_churn[['tenure_months','has_unlimited_data','has_premium_tech_support','has_device_protection','total_monthly_fee']] 
y = df_fin_churn['churn_label'].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

log_reg = LogisticRegression(max_iter=300)
log_reg_model = log_reg.fit(X_train, y_train)

y_pred = log_reg_model.predict_proba(X_test)[:,1]


auc = roc_auc_score(y_test,y_pred)
print('AUC:', auc)

In [ ]:
# # Test with XGboost with standard parameters
X = df_fin_churn[['tenure_months','has_unlimited_data','has_premium_tech_support','has_device_protection','total_monthly_fee']] 
y = df_fin_churn['churn_label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


xgboost_model = XGBClassifier(learning_rate=0.1,random_state=5)
xgboost_model.fit(X_train, y_train)

y_pred_proba = xgboost_model.predict_proba(X_test)[:,1]
auc_score = metrics.roc_auc_score(y_test, y_pred_proba)
print('AUC:', auc_score)

In [ ]:
# # Test with XGboost with suggested hyperparameter tested parameters
X = df_fin_churn[['tenure_months','has_unlimited_data','has_premium_tech_support','has_device_protection','total_monthly_fee']] 
y = df_fin_churn['churn_label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

xgboost_model = XGBClassifier(n_estimators = 500, max_depth=3, learning_rate = 0.01, random_state=5)
xgboost_model.fit(X_train, y_train)

y_pred_proba = xgboost_model.predict_proba(X_test)[:,1]
auc_score = metrics.roc_auc_score(y_test, y_pred_proba)
print('AUC:', auc_score)

In [ ]:
#Test with CatBoost


X = df_fin_churn[['tenure_months','has_unlimited_data','has_premium_tech_support','has_device_protection','total_monthly_fee']] 
y = df_fin_churn['churn_label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)
catboost_model = CatBoostClassifier(learning_rate= 0.01, depth=3, random_state=5)
catboost_model.fit(X_train, y_train, verbose=False)



In [ ]:
y_pred_proba = catboost_model.predict_proba(X_test)[:,1]
auc_score = metrics.roc_auc_score(y_test, y_pred_proba)
print('AUC:', auc_score)


In [ ]:
#Test with CatBoost with higher depth, learning rate and random_state



X = df_fin_churn[['tenure_months','has_unlimited_data','has_premium_tech_support','has_device_protection','total_monthly_fee']] 
y = df_fin_churn['churn_label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)
catboost_model = CatBoostClassifier(learning_rate= 0.1, depth=5, random_state=10)
catboost_model.fit(X_train, y_train, verbose=False)
                  
                  


In [ ]:
y_pred_proba = catboost_model.predict_proba(X_test)[:,1]
auc_score = metrics.roc_auc_score(y_test, y_pred_proba)
print('AUC:', auc_score)


Conclusion: Use Catboost with parameters: learning_rate= 0.01, depth=3, random_state=5

In [ ]:
X = df_fin_churn[['tenure_months','has_unlimited_data','has_premium_tech_support','has_device_protection','total_monthly_fee']] 
y = df_fin_churn['churn_label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

catboost_model_final = CatBoostClassifier(learning_rate= 0.01, depth=3, random_state=5)

from pathlib import Path

import joblib

joblib.dump(catboost_model_final, '../model/catboost_model_final.pkl')
